In [1]:
import yfinance as yf

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import statsmodels.api as sm

In [5]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

In [6]:
stock = 'adbe'

cutoff=0.51
invest = 100
years_of_data_to_process = 17

In [7]:
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")
start_year =  int(year) - years_of_data_to_process
start = datetime.datetime(start_year, 1, 1)

In [8]:
data =  yf.download(stock, start=start)

[*********************100%***********************]  1 of 1 completed


In [9]:
data

Open        High         Low       Close   Adj Close  \
Date                                                                     
2004-01-02   19.820000   19.850000   19.434999   19.549999   19.518400   
2004-01-05   19.650000   19.990000   19.230000   19.900000   19.867832   
2004-01-06   19.920000   19.990000   19.570000   19.920000   19.887802   
2004-01-07   19.719999   19.850000   19.555000   19.719999   19.694550   
2004-01-08   19.705000   19.940001   18.950001   19.000000   18.975479   
...                ...         ...         ...         ...         ...   
2021-05-27  505.000000  505.000000  497.450012  498.279999  498.279999   
2021-05-28  506.079987  511.369995  504.190002  504.579987  504.579987   
2021-06-01  504.540009  506.000000  492.309998  495.910004  495.910004   
2021-06-02  495.179993  501.820007  493.089996  495.769989  495.769989   
2021-06-03  492.399994  496.600006  487.339996  493.140015  493.140015   

              Volume  
Date                  
2004-01-02   3699600  
2004-01-05   9367400  
2004-01-06   5390400  
2004-01-07   5457800  
2004-01-08  13034800  
...              ...  
2021-05-27   2955600  
2021-05-28   1924800  
2021-06-01   2189000  
2021-06-02   1761300  
2021-06-03   1486218  

[4385 rows x 6 columns]

In [10]:
#data.drop(data.loc[data['Open']==0].index, inplace=True)
#data.drop(data.loc[data['Volume']==0].index, inplace=True)

In [11]:
df = data["Close"].pct_change() * 100

In [12]:
df = df.rename("Today_Change_%")
df = df.reset_index()
#df

In [13]:
df1 = pd.merge(data,df, on="Date")
#df1

In [14]:
df1.insert(7,'Volume_Lag', None)
df1['Volume_Lag'] = df1.Volume.shift(1).values/1000000000
df1.Volume = df1.Volume/1000000000

In [15]:
df1["Trend"] =  (df1["Close"] - df1["Low"])/ ((df1["High"] - df1["Low"]))

In [16]:
df1["Trend_Lag"] = df1["Trend"].shift(1)

In [17]:
df1['Close-Open'] =  df1['Close'] - df1['Open']
df1['Close-Open_Lag'] = df1['Close-Open'].shift(1)

In [18]:
df1['High-Low'] =  df1['High'] - df1['Low']
df1['High-Low_Lag'] = df1['High-Low'].shift(1)

In [19]:
df1 = sm.add_constant(df1)
df1["Up_Down"] = [1 if (i > 0) else 0 for i in df1["Today_Change_%"]]

In [20]:
df1.dropna(inplace= True)
#df1

In [21]:
X = df1[['const','Trend_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]

In [22]:
y = df1["Up_Down"].values

In [23]:
model = sm.Logit(y,X)

In [24]:
result =  model.fit()

Optimization terminated successfully.
         Current function value: 0.688336
         Iterations 5


In [25]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                 4384
Model:                          Logit   Df Residuals:                     4379
Method:                           MLE   Df Model:                            4
Date:                Thu, 03 Jun 2021   Pseudo R-squ.:                0.004567
Time:                        14:23:14   Log-Likelihood:                -3017.7
converged:                       True   LL-Null:                       -3031.5
Covariance Type:            nonrobust   LLR p-value:                 1.443e-05
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.4506      0.084      5.334      0.000       0.285       0.616
Trend_Lag         -0.3942      0.115     -3.437      0.001      -0.619      -0.169
Close-Open_Lag    -0.0088      0.013     -0.685      0.493      -0.034       0.016
High-Low_Lag      -0.0002      0.008     -0.027      0.978      -0.016       0.015
Volume_Lag       -25.4500      8.479     -3.001      0.003     -42.069      -8.831
==================================================================================
"""

In [26]:
prediction = result.predict(X)

In [27]:
df1['Prediction_Caculated'] = pd.array(prediction)
df1['Prediction_indicator'] = pd.array([1 if i > cutoff else 0 for i in prediction])
#df1.tail(10)

In [28]:
y = df1["Up_Down"].values

In [29]:
def confusion_matrix(act,pred):
    predtrans = ['Up' if i > cutoff else 'Down' for i in pred]
    actuals = ['Up' if i > 0 else 'Down' for i in act]
    confusion_matrix = pd.crosstab(pd.Series(actuals),
                                   pd.Series(predtrans),
                                   rownames = ["Actual"],
                                   colnames = ["Predict"]
                                  )
    return confusion_matrix

In [30]:
confusion_matrix(y,prediction)

Predict  Down    Up
Actual             
Down      738  1328
Up        723  1595

In [31]:
z = confusion_matrix(y,prediction)
try:
    print((z.loc['Down','Down'] + z.loc['Up','Up']) / len(df1))
except:
    pass

0.5321624087591241


In [32]:
try:
    print( (z.loc['Down', 'Down']+ z.loc['Up','Up']) / (z.loc['Down', 'Down']+ z.loc['Up','Up'] + z.loc['Down','Up']) )
except:
    pass

0.6372575798962032


In [33]:
df1 = df1.assign(share=np.nan,money=np.nan)
#df1

In [34]:
diff_years = round((df1.iloc[-1,1] - df1.iloc[0,1])/np.timedelta64(1,'Y') + 0.5)   

In [35]:
#Simulate Investment transaction buy on opening when predict UP and sell daily average when predict DOWN

def buy_sell(open_price, sell_price,prediction, money, share):
    if prediction == 1 and money != 0:
        share =  money / open_price
        money = 0
    elif prediction == 0 and share != 0:
        money = share * sell_price
        share = 0
    else: pass
    return [money, share]
money = invest
share = 0
for i in range(len(df1)):
    [money, share] = buy_sell(df1.iloc[i,2],(df1.iloc[i,3]+df1.iloc[i,4])/2,df1.iloc[i,18], money, share)
    df1.iloc[i,19] = share
    df1.iloc[i,20] = money
    

In [36]:
#df1

In [37]:
print("\nIf ${:,.2f} was invested in [ {} ], and Just Hold and Not Trade  for {:,} years, the ROI = ${:,.2f}".format( invest, stock.upper(), diff_years, invest/data.iloc[0, 0] * data.iloc[-1,0]))


If $100.00 was invested in [ ADBE ], and Just Hold and Not Trade  for 18 years, the ROI = $2,484.36


In [38]:
print ("\nIf ${:,.2f} was invested {:2d} years ago, buy and sell according this script\'s recommandation, the ROI = ${:,.2f}".format(invest, diff_years, (money + (share * df1.iloc[-1,6]))))


If $100.00 was invested 18 years ago, buy and sell according this script's recommandation, the ROI = $1,017.77


In [39]:
#df1.tail(10)

In [40]:
df1_summary=df1[['Date', 'Up_Down','Prediction_indicator']].copy()
df1_summary['Stock Market Performance'] = df1_summary['Up_Down'].apply(lambda x: 'Up' if x > 0 else 'Down')
df1_summary['Scribe Predection'] = df1_summary['Prediction_indicator'].apply(lambda x: 'Up' if x > 0 else 'Down')
print (df1_summary[['Date','Stock Market Performance','Scribe Predection']].tail(15))

           Date Stock Market Performance Scribe Predection
4370 2021-05-13                       Up                Up
4371 2021-05-14                       Up                Up
4372 2021-05-17                     Down              Down
4373 2021-05-18                     Down                Up
4374 2021-05-19                     Down                Up
4375 2021-05-20                       Up              Down
4376 2021-05-21                     Down                Up
4377 2021-05-24                       Up                Up
4378 2021-05-25                       Up                Up
4379 2021-05-26                       Up              Down
4380 2021-05-27                     Down                Up
4381 2021-05-28                       Up                Up
4382 2021-06-01                     Down                Up
4383 2021-06-02                     Down                Up
4384 2021-06-03                     Down                Up


In [41]:
print ("\nToday actually goes up,", end = ' ') if (df1.iloc[-1,16] == 1) else print ("\nToday actually goes down.", end = " ")
print ("Base on yesterday\'s data, ", end = '')
print ("Today\'s Predication is going up.") if (df1.iloc[-1,18] == 1) else print ("Today\'s Predication is going down")
print ("Actual and Predication MATCH") if (df1.iloc[-1,16] == df1.iloc[-1,18]) else print("Actual and Predication DO NOT match")


Today actually goes down. Base on yesterday's data, Today's Predication is going up.
Actual and Predication DO NOT match


In [42]:
x_tran= df1[df1.Date.dt.year < 2021][['const','Trend_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]
y_train=df1[df1.Date.dt.year < 2021]["Up_Down"]
x_test= df1[df1.Date.dt.year >= 2021][['const','Trend_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]
y_test= df1[df1.Date.dt.year >= 2021]["Up_Down"]

In [43]:
model = sm.Logit(y_train,x_tran)

result=model.fit()

Optimization terminated successfully.
         Current function value: 0.688160
         Iterations 5


In [44]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Up_Down   No. Observations:                 4279
Model:                          Logit   Df Residuals:                     4274
Method:                           MLE   Df Model:                            4
Date:                Thu, 03 Jun 2021   Pseudo R-squ.:                0.004840
Time:                        14:23:16   Log-Likelihood:                -2944.6
converged:                       True   LL-Null:                       -2959.0
Covariance Type:            nonrobust   LLR p-value:                 9.227e-06
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.4498      0.085      5.283      0.000       0.283       0.617
Trend_Lag         -0.3880      0.116     -3.350      0.001      -0.615      -0.161
Close-Open_Lag    -0.0131      0.014     -0.946      0.344      -0.040       0.014
High-Low_Lag       0.0005      0.009      0.054      0.957      -0.016       0.017
Volume_Lag       -25.9961      8.528     -3.048      0.002     -42.711      -9.281
==================================================================================
"""

In [45]:
prediction = result.predict(x_test)
confusion_matrix(y_test, prediction)

Predict  Down  Up
Actual           
Down       15  34
Up         15  41

In [46]:
z = confusion_matrix(y_test,prediction)
try:
    print((z.loc['Down','Down'] + z.loc['Up','Up']) / len(x_test))
except:
    pass

0.5333333333333333


In [47]:
# Base on today's data to  predicate tomorrow trend
now_up_down  = result.predict([1.0, df1.iloc[-1, 10], df1.iloc[-1, 12], df1.iloc[-1, 14], df1.iloc[-1, 7]])
print ("\nCurrent trend = %.4f,  " %now_up_down, end=' ')
print ("%s will go up!" %stock.upper()) if now_up_down > cutoff else print ("%s will go down!" %stock.upper()) 


Current trend = 0.5406,   ADBE will go up!


In [48]:
#df1.to_csv('fb.csv', index = False)